In [21]:
import requests_cache

import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import date, timedelta
import os
import time
from datetime import datetime
from dateutil import parser
from datetime import datetime

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

In [7]:
requests_cache.install_cache('demo_cache',expire_after=None,allowable_methods=['GET'])
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\MyPC\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
C:\Users\MyPC\AppData\Local\Temp/ipykernel_8396/3689447328.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [8]:
link = "https://www.worldometers.info/coronavirus/"
driver.get(link)

In [9]:
def split_list(a_list):
    #The number of countries is steady
    return a_list[:226], a_list[226:]

In [10]:
row_world = driver.find_elements(By.XPATH,"//tr[@class='total_row_world odd']")
row_total = driver.find_elements(By.XPATH,"//tr[@class='even'] | //tr[@class='odd']")

rows_today = []
rows_past = []
rows_past_modify = []
rows_yesterday = []
rows_two_days_ago = []
count = 0
row_count = 224

for n in row_total:
    n = n.get_attribute('innerText')
    if count == row_count:
        rows_past.append(n.split("\n"))
    else:
        rows_today.append(n.split("\t"))
        count+=1

for n in rows_past:
    n.pop(0)
    n = n[: len(n) - 6]
    rows_past_modify.append(n)

rows_yesterday,rows_two_days_ago = split_list(rows_past_modify)
rows_yesterday.pop(0)

#Get date
today = date.today()
yesterday = (today - timedelta(1)).strftime("%d-%m-%Y")
two_days_ago = (today - timedelta(2)).strftime("%d-%m-%Y")
now = datetime.now()
cur_time = int(now.strftime('%H'))
#Add date to df
for row in rows_yesterday:
    row.insert(1,yesterday)
for row in rows_two_days_ago:
    row.insert(1,two_days_ago)

covid_data = rows_two_days_ago + rows_yesterday

for i in covid_data:
    if len(i) != len(header_by_day):
        print("Error Header in #",i[0],i[2])
        print("Trying to fix...")
        time.sleep(5)
        i.pop(-1)

In [11]:
header_by_day = ["#","Date","Country,Other","TotalCases","NewCases","TotalDeaths","NewDeaths","TotalRecovered","NewRecovered",
"ActiveCases","Serious,Critical","Tot Cases/1M pop","Deaths/1M pop","TotalTests","Tests/1M pop","Population","Continent"]

header_by_week = ["#", "Country,Other", "Cases in the last 7 days", "Cases in the preceding 7 days", 
"Weekly case /%/ change", "Cases in the last 7 days/1M pop", "Deaths in the last 7 days", 
"Deaths in the preceding 7 days", "Weekly Death /%/ change", "Deaths in the last 7 days/1M pop", "Population", "Continent"]

In [24]:
for i in covid_data:
    if len(i) != len(header_by_day):
        print("Error Header in #",i[0],i[2])
        print("Trying to fix...")
        time.sleep(5)
        i.pop(-1)

In [25]:
df_covid = pd.DataFrame(data=rows_yesterday,columns=header_by_day)

In [25]:
if not os.path.isfile('covid-data-test/Corona_by_day.tsv'):
    df_covid = pd.DataFrame(data=covid_data,columns=header_by_day)
    df_covid.to_csv("covid-data-test/Corona_by_day.tsv", sep = "\t", mode = "a", index = False)
else:
    df_covid.to_csv("covid-data/Corona_by_day.tsv", sep = "\t", mode = "a", index = False,header=False)

In [7]:
link = "https://www.worldometers.info/coronavirus/weekly-trends/#weekly_table"
driver.get(link)
row_total = driver.find_elements(By.XPATH,"//tr[@class='even'] | //tr[@class='odd']")

In [8]:
for n in row_total:
    n = n.get_attribute('textContent').split("\n")
    n[:] = [x.strip() for x in n if x.strip()]
    if len(n) != len(header_by_week):
        print("Error Header in",n)
        

In [9]:
row_week_trend = []

for n in row_total:
    n = n.get_attribute('textContent').split("\n")
    n[:] = [x.strip() for x in n if x.strip()]
    row_week_trend.append(n)
    #if len(n) != len(header_by_week):
        #print("Error Header in",n)

In [13]:
df_covid_by_week = pd.DataFrame(data=row_week_trend,columns=header_by_week)

In [14]:
df_covid_by_week.to_csv("covid-data/Corona_by_week.tsv", sep = "\t", mode = "w", index = False)